In [1]:
import pandas as pd
import os

In [9]:
combined_df = {}

In [10]:
# sheet_name = '1、初试'
# title_row = 3
# sheet_name = '2、复试'
# title_row = 4
# sheet_name = '3、录用'
# title_row = 4
sheet_name = '4、签约'
title_row = 4

In [11]:
skip=[i for i in range(title_row-1)]
folder_path = os.getcwd()
output_file = f'{sheet_name}.xlsx' 

print(f'skip:\t\t{skip}')
print(f'folder_path:\t{folder_path}')
print(f'output_file:\t{output_file}')

skip:		[0, 1, 2]
folder_path:	c:\Users\long\Desktop\0
output_file:	4、签约.xlsx


In [12]:
sheet_data = pd.DataFrame()
count = 0

for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            file_path = os.path.join(folder_path, file_name)
            try:
                df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skip)
                sheet_data = pd.concat([sheet_data, df], ignore_index=True)
                count += 1
            except PermissionError:
                print(f"无法访问文件: {file_name}")
            except ValueError:
                print(f"文件 {file_name} 中没有找到页签: {sheet_name}")
            except (FileNotFoundError, OSError):
                print(f"文件 {file_name} 无法打开或不存在")
            except Exception as e:
                print(f"读取文件 {file_name} 时发生未知错误: {e}")
combined_df[sheet_name] = sheet_data
print(f'文件数量：\t{count}')

c:\Users\long\anaconda3\envs\rnd-llm-excel\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\long\anaconda3\envs\rnd-llm-excel\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\long\anaconda3\envs\rnd-llm-excel\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\long\anaconda3\envs\rnd-llm-excel\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\long\AppData\Local\Temp\ipykernel_31960\3634412212.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the resul

文件 2、复试.xlsx 中没有找到页签: 4、签约
文件 3、录用.xlsx 中没有找到页签: 4、签约
文件 汇总.xlsx 中没有找到页签: 4、签约
文件数量：	23


c:\Users\long\anaconda3\envs\rnd-llm-excel\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [13]:
combined_df.keys()

dict_keys(['4、签约'])

In [14]:
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in combined_df.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
print(f"已成功合并到{output_file}中")

已成功合并到4、签约.xlsx中


In [2]:
from openpyxl import load_workbook, Workbook
from copy import copy
import os
import pandas as pd

# 指定文件路径和工作表名称
folder_path = '学校报表'
sheet_names = ['1、初试', '2、复试', '3、录用', '4、签约']
output_file = 'a.xlsx'

In [3]:
excel_files = [f for f in os.listdir(folder_path) if f.endswith(('.xlsx', '.xls'))]
count_files = len(excel_files)

In [4]:
def get_sheet_names(file_path):
    """从指定的 Excel 文件中获取所有工作表的名称"""
    try:
        # 加载 Excel 文件
        wb = load_workbook(file_path, read_only=True)  # 使用 read_only=True 提高性能
        sheet_names = wb.sheetnames  # 获取所有工作表名称
        wb.close()  # 关闭工作簿
        return sheet_names
    except FileNotFoundError:
        print(f"文件 {file_path} 未找到！")
        return []
    except Exception as e:
        print(f"无法读取文件: {e}")
        return []

In [ ]:
combined_df = {}
title_row_dict = {}

for sheet_name in sheet_names:
    sheet_data = pd.DataFrame()
    for idx, file_name in enumerate(excel_files):  # 遍历所有excel
        file_path = os.path.join(folder_path, file_name)
        if idx == 0: # 第一次
            df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, dtype=str)
            title_row = None
            for i, row in df.iterrows():
                if row.notnull().all():  # 检测非空行作为标题行
                    title_row = i + 1
                    break
            if title_row == None:
                raise Exception('没有找到标题行，请手动设置！')
            title_row_dict[sheet_name] = title_row
            # new_columns = df.iloc[title_row-1].values
            # df.columns = new_columns
            df.columns = range(len(df.columns))
            df = df.iloc[title_row:].reset_index(drop=True)
            sheet_data = pd.concat([sheet_data, df], ignore_index=True, join='outer')
            # if df.columns.duplicated().any():
            #     raise Exception(f"文件 {file_name} 中存在重复列名，索引: {idx}")
        else:
            df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=title_row_dict[sheet_name]-1, dtype=str)
            df.columns = range(len(df.columns))
            sheet_data = pd.concat([sheet_data, df], ignore_index=True, join='outer')
    combined_df[sheet_name] = sheet_data

In [9]:
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in combined_df.items():
        title_row = title_row_dict[sheet_name]
        df.to_excel(writer, sheet_name=sheet_name, header=False, index=False)
        
        # 获取 workbook 和 worksheet 对象
        new_wb = writer.book
        new_ws = new_wb[sheet_name]
        new_ws.insert_rows(1, title_row) 
        
        # 获取原始 Excel 文件的标题格式
        original_wb = load_workbook(os.path.join(folder_path, excel_files[0]), data_only=True)
        original_ws = original_wb[sheet_name]

        for row in original_ws.iter_rows(min_row=1, max_row=title_row, values_only=False):
            for cell in row:
                new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
                if cell.has_style:
                    new_cell.font = copy(cell.font)
                    new_cell.fill = copy(cell.fill)
                    new_cell.alignment = copy(cell.alignment)
                    new_cell.border = copy(cell.border)
                    new_cell.number_format = cell.number_format

        for merged_range in original_ws.merged_cells.ranges:
            min_col, min_row, max_col, max_row = merged_range.bounds
            if min_row <= title_row:
                new_ws.merge_cells(start_row=min_row, start_column=min_col,
                                end_row=max_row, end_column=max_col)
                
original_wb.close()
new_wb.close()

In [ ]:
def copy_title(new_ws, ws, title_row): # 复制标题（标题行之前）
    for row in ws.iter_rows(min_row=1, max_row=title_row, values_only=False): # 复制原始工作表的标题行
        for cell in row:
            new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.fill = copy(cell.fill)
                new_cell.alignment = copy(cell.alignment)
                new_cell.border = copy(cell.border)
                new_cell.number_format = cell.number_format
    for merged_range in ws.merged_cells.ranges: # 复制合并单元格信息
        min_col, min_row, max_col, max_row = merged_range.bounds
        if min_row <= title_row:
            new_ws.merge_cells(start_row=min_row, start_column=min_col,
                            end_row=max_row, end_column=max_col)

In [ ]:
def copy_data(new_ws, ws, title_row): # 复制数据行（从标题行之后开始）
    data_start_row = title_row + 1
    for r_idx, row in enumerate(ws.iter_rows(min_row=data_start_row, values_only=True), start=new_ws.max_row):
        for c_idx, cell in enumerate(row, start=1):
            new_ws.cell(row=r_idx, column=c_idx, value=cell)

In [ ]:
# 复制原始工作表的标题行
for row in original_ws.iter_rows(min_row=1, max_row=title_row, values_only=False):
    for cell in row:
        new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
        if cell.has_style:
            new_cell.font = copy(cell.font)
            new_cell.fill = copy(cell.fill)
            new_cell.alignment = copy(cell.alignment)
            new_cell.border = copy(cell.border)
            new_cell.number_format = cell.number_format

# 复制合并单元格信息
for merged_range in original_ws.merged_cells.ranges:
    min_col, min_row, max_col, max_row = merged_range.bounds
    if min_row <= title_row:
        new_ws.merge_cells(start_row=min_row, start_column=min_col,
                           end_row=max_row, end_column=max_col)

In [34]:
# 创建一个新的工作簿并添加工作表
new_wb = Workbook()
new_ws = new_wb.create_sheet(title=sheet_name)

In [35]:
new_ws.max_row

1

In [2]:
# 加载原始 Excel 文件
original_wb = load_workbook(file_path, data_only=True)
original_ws = original_wb[sheet_name]

In [ ]:
# 设置 DataFrame 的列名并清理数据
new_columns = df.iloc[title_row-1].values
df.columns = new_columns
df = df.iloc[title_row:].reset_index(drop=True)

In [22]:
for merged_range in original_ws.merged_cells:
    print(merged_range)

A2:F2
A1:AM1
AJ2:AM2
G2:AI2


In [40]:
# 复制原始工作表的标题行
for row in original_ws.iter_rows(min_row=1, max_row=title_row, values_only=False):
    for cell in row:
        new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
        if cell.has_style:
            new_cell.font = copy(cell.font)
            new_cell.fill = copy(cell.fill)
            new_cell.alignment = copy(cell.alignment)
            new_cell.border = copy(cell.border)
            new_cell.number_format = cell.number_format

# 复制合并单元格信息
for merged_range in original_ws.merged_cells.ranges:
    min_col, min_row, max_col, max_row = merged_range.bounds
    if min_row <= title_row:
        new_ws.merge_cells(start_row=min_row, start_column=min_col,
                           end_row=max_row, end_column=max_col)
        
# 将 DataFrame 的数据追加到新工作表的后面
# for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), start=title_row+1):
#     for c_idx, value in enumerate(row, start=1):
#         new_ws.cell(row=r_idx, column=c_idx, value=value)

# 复制数据行（从标题行之后开始）
data_start_row = title_row + 1
for r_idx, row in enumerate(original_ws.iter_rows(min_row=data_start_row, values_only=False), start=data_start_row):
    for c_idx, cell in enumerate(row, start=1):
        new_ws.cell(row=r_idx, column=c_idx, value=cell.value)

# 保存新的 Excel 文件
new_wb.save('a.xlsx')

# 关闭原始工作簿
original_wb.close()

# 关闭新工作簿（可选，但建议保留以确保资源释放）
new_wb.close()
